In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import os 
import numpy as np
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelBinarizer
from sklearn.svm import SVR
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from collections import Counter

In [27]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mukul/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [28]:
dirs = [files for files in glob.glob('./Generated CSV/*.csv')]

In [29]:
def create_feature(sentence,line_number,all_n_word_tokens,all_pos_tags):
    features = {
        'n_word_tokens': all_n_word_tokens[line_number],
        'standard POS tags as in Penn Treebank': all_pos_tags[line_number],
        'line number in conversation': line_number ,
        ## 'utterance type ' : ?? ##I think we should give that rsa tags that we had with each sentence , or make some classification with it,
        'utterance length':len(word_tokenize(sentence)),
        'n word tokens in previous & next 5 utterances' : all_n_word_tokens[line_number-5: line_number+5],
         'standard POS tags in previous & next 5 utterances': all_pos_tags[line_number-5:line_number+5],
        
    }
    ##n word tokens 

    return ##dictionary flatenned

In [30]:
def word_token_generator(df):
    counter = Counter()
    for index,lines in enumerate(df['Conversation']):
        tokens = word_tokenize(lines)
        for word in tokens:
            counter[word]+=1
    return counter

In [31]:
def all_nword_generator(df,counter):
    all_nword = CustomList([])
    for line in df['Conversation']:
        all_nword.append([counter[word] for word in line])
    return all_nword
    

In [32]:
class CustomList(list):
    def __getitem__(self, n):
        if n >= len(self) or n < 0 :
            return []
        return super(CustomList, self).__getitem__(n)
    

def all_pos_tag_generator(df):
    all_pos = CustomList([])
    for line in df['Conversation']:
        temp = Counter(pos_tag(line))
        all_pos.append(temp)
    return all_pos

a
0


In [ ]:



def valid_text(conversations):
    X = []
    Y = []
    for conversation_file in conversations:
        df = pd.read_csv(conversation_file,delimiter=',')
        df = df[df['Conversation'].notnull()]
        counter = word_token_generator(df)
        all_n_word_tokens = all_nword_generator(df,counter)
        all_pos_tags = all_pos_tag_generator(df) 
        for index,lines in enumerate(df['Conversation']):
#             print(df[df.columns[0]])
            X.append(create_feature(lines,df[df.columns[0]],all_n_word_tokens,all_pos_tags))
            target.append(float(df['Class'][index]))
    
    return text,target

In [ ]:
text,target = valid_text(dirs)

In [412]:
X_train, X_test, y_train, y_test = train_test_split(text, target, test_size=0.1,shuffle=False)

In [255]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    embeddings_index = {}
    f = open(r'../glove.840B.300d.txt', encoding='utf8')
    for line in f:
        values = line.split()
        word = ''.join(values[:-300])
        coefs = np.asarray(values[-300:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

    

In [ ]:
path = '../glove.840B.300d.txt'
w2v_dict = loadGloveModel(path)

In [413]:
def w2v_mapper(data):
    a = []
    for sentences in tqdm(data):
        sentence = word_tokenize(sentences)
        array = []
        for word in sentence:
            try:
                array.append([w2v_dict[word]])
            except:
                pass
        
        w2v_sentence = np.mean(array, axis=0)
        a.append(w2v_sentence.flatten())
        
    a = np.array(a)
    return a

In [414]:
X_train_emb = w2v_mapper(X_train)

100%|██████████| 11284/11284 [00:01<00:00, 5906.88it/s]


In [415]:
X_test_emb = w2v_mapper(X_test)

100%|██████████| 1254/1254 [00:00<00:00, 5884.73it/s]


In [406]:
def reduce_dim(data):
    pca = PCA(n_components=200)
    result = pca.fit_transform(data)
    return result

In [407]:
X_train_pca = reduce_dim(X_train_emb)
X_test_pca = reduce_dim(X_test_emb)

In [420]:
clf = SGDClassifier(loss='hinge', penalty='l1',
                          alpha=1e-4, random_state=42,
                          max_iter=250, tol=None).fit(X_train_emb,y_train)


In [422]:
clf_log = LogisticRegression(solver='lbfgs',multi_class='multinomial',max_iter=10000).fit(X_train_emb,y_train)


In [421]:
predicted = clf.predict(X_test_emb)
np.mean(predicted == y_test) 

0.3572567783094099

In [271]:
X_train_emb.shape

(11284, 300)

In [423]:
predicted = clf_log.predict(X_test_emb)
np.mean(predicted == y_test) 

0.3572567783094099

In [322]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import Callback

Using TensorFlow backend.


In [359]:
encoder = LabelBinarizer()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [391]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=300))
model.add(Dense(3, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_emb, y_train, epochs=100, batch_size=64, verbose=2)

Epoch 1/100
 - 0s - loss: 1.0734 - accuracy: 0.4127
Epoch 2/100
 - 0s - loss: 1.0644 - accuracy: 0.4288
Epoch 3/100
 - 0s - loss: 1.0576 - accuracy: 0.4421
Epoch 4/100
 - 0s - loss: 1.0513 - accuracy: 0.4484
Epoch 5/100
 - 0s - loss: 1.0447 - accuracy: 0.4537
Epoch 6/100
 - 0s - loss: 1.0392 - accuracy: 0.4538
Epoch 7/100
 - 0s - loss: 1.0329 - accuracy: 0.4613
Epoch 8/100
 - 0s - loss: 1.0274 - accuracy: 0.4702
Epoch 9/100
 - 0s - loss: 1.0230 - accuracy: 0.4785
Epoch 10/100
 - 0s - loss: 1.0172 - accuracy: 0.4790
Epoch 11/100
 - 0s - loss: 1.0118 - accuracy: 0.4845
Epoch 12/100
 - 0s - loss: 1.0044 - accuracy: 0.4928
Epoch 13/100
 - 0s - loss: 0.9975 - accuracy: 0.4944
Epoch 14/100
 - 0s - loss: 0.9928 - accuracy: 0.4982
Epoch 15/100
 - 0s - loss: 0.9897 - accuracy: 0.4988
Epoch 16/100
 - 0s - loss: 0.9813 - accuracy: 0.5077
Epoch 17/100
 - 0s - loss: 0.9764 - accuracy: 0.5121
Epoch 18/100
 - 0s - loss: 0.9713 - accuracy: 0.5155
Epoch 19/100
 - 0s - loss: 0.9676 - accuracy: 0.5181
Ep

In [392]:
score = model.evaluate(X_test_emb, y_test, batch_size=128, verbose=2)


In [393]:
score

[1.5679596909685758, 0.35247209668159485]